In [10]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy.ndimage import uniform_filter1d
from tqdm import tqdm
import joblib

In [11]:
SOURCE_DIR = "data/preprocessed_ml"   # change per shot
OUT_DIR    = "data/features_scored_4levels"  # outputs go here
FS         = 2148.1481
WELCH_NPERSEG = 512
MIN_NUM_SAMPLES = 50

# Fixed channel canonical order - set names exactly matching CSV column names (case sensitive)
# Edit this list to match the exact column names in your preprocessed CSVs.
CANONICAL_CHANNEL_ORDER = [
    'Rectus Femoris right', 'Rectus Femoris left', 
    'Hamstrings right', 'Hamstrings left', 
    'TibilaisÂ Anterior right', 'TibilaisÂ Anterior left', 
    'Gastrocnemius right', 'Gastrocnemius left'
]

# RMS window (seconds) for moving-RMS
RMS_WINDOW_MS = 50
RMS_WINDOW_SAMPLES = max(1, int((RMS_WINDOW_MS/1000.0) * FS))

# Save settings
os.makedirs(OUT_DIR, exist_ok=True)
print("SOURCE_DIR:", SOURCE_DIR)
print("OUT_DIR:", OUT_DIR)
print("FS:", FS, "RMS window samples:", RMS_WINDOW_SAMPLES)



SOURCE_DIR: data/preprocessed_ml
OUT_DIR: data/features_scored_4levels
FS: 2148.1481 RMS window samples: 107


In [12]:
# Cell 2 - helpers: safe PSD, move-RMS, canonicalize columns
def safe_welch(x, fs=FS, nperseg=WELCH_NPERSEG):
    nperseg_eff = min(len(x), max(16, nperseg))
    try:
        f, Pxx = welch(x, fs=fs, nperseg=nperseg_eff)
    except Exception:
        f = np.array([0.0])
        Pxx = np.array([0.0])
    return f, Pxx

def moving_rms(x, window_samples=RMS_WINDOW_SAMPLES):
    if len(x) < window_samples or window_samples <= 1:
        # fallback to global RMS
        return np.sqrt(np.mean(x**2)) * np.ones_like(x)
    sq = x.astype(float)**2
    # uniform_filter1d behaves as a sliding window mean; take sqrt for RMS
    mean_sq = uniform_filter1d(sq, size=window_samples, mode='nearest')
    return np.sqrt(mean_sq)

def canonicalize_emg_df(df, canonical_order=CANONICAL_CHANNEL_ORDER):
    """
    Return DataFrame with columns in canonical order.
    If a canonical column is missing, create it filled with NaNs.
    If there are extra columns, append them at the end.
    """
    cols_present = df.columns.tolist()
    ordered = []
    for c in canonical_order:
        if c in cols_present:
            ordered.append(c)
        else:
            # create missing column as NaN
            df[c] = np.nan
            ordered.append(c)
    # append any remaining columns not in canonical list (to avoid data loss)
    remaining = [c for c in cols_present if c not in canonical_order]
    ordered += remaining
    return df[ordered]


In [13]:
# Cell 3 - per-channel feature extractors
def extract_time_features(x):
    x = np.asarray(x).astype(float)
    if x.size == 0:
        return {
            "mean": np.nan, "std": np.nan, "rms": np.nan,
            "mav": np.nan, "wl": np.nan, "peak": np.nan, "iEMG": np.nan
        }
    return {
        "mean": float(np.mean(x)),
        "std":  float(np.std(x)),
        "rms":  float(np.sqrt(np.mean(x**2))),
        "mav":  float(np.mean(np.abs(x))),
        "wl":   float(np.sum(np.abs(np.diff(x)))),
        "peak": float(np.max(x)),
        "iEMG": float(np.trapz(np.abs(x)))  # numeric integral
    }

def extract_freq_features(x, fs=FS):
    x = np.asarray(x).astype(float)
    if len(x) < 4:
        return {"mnf": np.nan, "mdf": np.nan, "bp_20_60": np.nan, "bp_60_100": np.nan, "bp_100_200": np.nan}
    f, Pxx = safe_welch(x, fs=fs)
    total = np.sum(Pxx) + 1e-12
    mnf = float(np.sum(f * Pxx) / total)
    csum = np.cumsum(Pxx)
    half = total / 2.0
    idx = np.searchsorted(csum, half)
    mdf = float(f[idx]) if idx < len(f) else float(f[-1])
    def bandpow(a,b):
        mask = (f >= a) & (f <= b)
        return float(np.trapz(Pxx[mask], f[mask])) if np.any(mask) else 0.0
    return {"mnf": mnf, "mdf": mdf, "bp_20_60": bandpow(20,60), "bp_60_100": bandpow(60,100), "bp_100_200": bandpow(100,200)}


In [14]:
# Cell 4 - single-file processing & feature assembly
def process_file_to_features(path):
    """
    Read one preprocessed CSV and return a dict of features + metadata.
    """
    df = pd.read_csv(path)
    df = canonicalize_emg_df(df)  # ensure canonical column order
    # detect time column
    time_cols = [c for c in df.columns if 'time' in c.lower() or 'timestamp' in c.lower()]
    time_col = time_cols[0] if time_cols else None
    emg_cols = [c for c in df.columns if c != time_col]
    n_samples = df.shape[0]

    row = {}
    row['file'] = path
    row['player'] = Path(path).parent.name
    row['n_samples'] = n_samples
    row['fs_used'] = FS

    # gather per-channel features for both envelope (assumed) and filtered-like values
    # we will compute both absolute and relative (per-trial normalized) metrics
    channel_means = {}
    channel_peaks = {}

    # first pass compute envelope/time-domain/freq features and moving RMS
    per_channel_data = {}
    for ch in emg_cols:
        x = pd.to_numeric(df[ch], errors='coerce').fillna(0).values.astype(float)
        per_channel_data[ch] = {}
        td = extract_time_features(x)
        per_channel_data[ch].update(td)
        ff = extract_freq_features(x)
        per_channel_data[ch].update(ff)
        # timing features using envelope
        peak_idx = int(np.nanargmax(x)) if np.any(~np.isnan(x)) else 0
        per_channel_data[ch]['time_to_peak_s'] = peak_idx / FS if n_samples>0 else np.nan
        # duration above halfmax
        halfmax = 0.5 * per_channel_data[ch].get('peak', np.nan)
        if np.isnan(halfmax):
            per_channel_data[ch]['dur_halfmax_s'] = np.nan
        else:
            per_channel_data[ch]['dur_halfmax_s'] = float(np.sum(x > halfmax) / FS) if n_samples>0 else np.nan
        # moving RMS
        mr = moving_rms(x, RMS_WINDOW_SAMPLES)
        per_channel_data[ch]['mrms_mean'] = float(np.mean(mr))
        per_channel_data[ch]['mrms_peak'] = float(np.max(mr))
        channel_means[ch] = per_channel_data[ch]['mean']
        channel_peaks[ch] = per_channel_data[ch]['peak']

    # compute per-trial normalization constants
    all_means = np.array([v for v in channel_means.values() if not np.isnan(v)])
    all_peaks = np.array([v for v in channel_peaks.values() if not np.isnan(v)])
    trial_mean = np.nanmean(all_means) if all_means.size>0 else 1.0
    trial_peak = np.nanmax(all_peaks) if all_peaks.size>0 else 1.0
    if trial_peak == 0:
        trial_peak = 1.0

    # fill row with structured names and also relative versions (divided by trial peak)
    for ch in emg_cols:
        chdata = per_channel_data[ch]
        for k,v in chdata.items():
            # convert keys like 'rms' to 'RF_L__rms'
            colname = f"{ch}__{k}"
            row[colname] = v
            # relative versions for amplitude-like features (rms, peak, iEMG, mrms_mean, mrms_peak)
            if k in ('rms','peak','iEMG','mrms_mean','mrms_peak','mav'):
                rel_name = f"{ch}__{k}_rel"
                row[rel_name] = v / trial_peak if (not np.isnan(v) and trial_peak!=0) else np.nan

    # symmetry & co-activation features using canonical pairs
    # attempt matching by channel base names: RF, TA, GAS, HAM
    def find_col(ch_list, pattern):
        # return first channel matching pattern
        for c in ch_list:
            if pattern.lower() in c.lower():
                return c
        return None

    # define expected muscle groups and left/right patterns
    muscles = ["Rectus Femoris", "TibilaisÂ Anterior", "Gastrocnemius", "Hamstrings"]
    for m in muscles:
        left = find_col(emg_cols, m + " l")
        right = find_col(emg_cols, m + " r")
        if left and right:
            meanL = per_channel_data[left].get('mean', np.nan)
            meanR = per_channel_data[right].get('mean', np.nan)
            row[f"{m}__LR_mean_ratio"] = meanL / (meanR + 1e-12) if not np.isnan(meanL) else np.nan
            row[f"{m}__LR_mean_absdev1"] = abs((meanL/ (meanR + 1e-12)) - 1.0)
            # co-activation index simple: overlap of normalized envelopes
            # read original arrays for indices (safe guard)
            try:
                xL = pd.to_numeric(df[left], errors='coerce').fillna(0).values.astype(float)
                xR = pd.to_numeric(df[right], errors='coerce').fillna(0).values.astype(float)
                # normalize by peak to get 0..1
                pL = np.max(xL) if np.max(xL)!=0 else 1.0
                pR = np.max(xR) if np.max(xR)!=0 else 1.0
                nL = xL / pL
                nR = xR / pR
                # simple coactivation = mean(min(nL,nR))
                coact = np.mean(np.minimum(nL, nR))
                row[f"{m}__LR_coact_simple"] = float(coact)
            except Exception:
                row[f"{m}__LR_coact_simple"] = np.nan

    # Add trial-level summary stats
    row['trial_mean_of_channel_means'] = float(trial_mean)
    row['trial_peak_of_channel_peaks'] = float(trial_peak)

    return row


In [15]:
# Cell 5 - iterate all files, collect features, save master CSV & QC
def find_csv_files(root_dir):
    fl = []
    for root, _, files in os.walk(root_dir):
        for f in files:
            if f.lower().endswith(".csv"):
                fl.append(os.path.join(root, f))
    return sorted(fl)

files = find_csv_files(SOURCE_DIR)
print("Found", len(files), "files.")
rows = []
qc_rows = []
for p in tqdm(files):
    try:
        feats = process_file_to_features(p)
        if feats is None:
            qc_rows.append({"file": p, "status": "no_emg"})
            continue
        rows.append(feats)
        qc_rows.append({"file": p, "status": "ok", "n_samples": feats.get('n_samples', np.nan)})
    except Exception as e:
        qc_rows.append({"file": p, "status": "error", "error": str(e)})
        print("Error processing", p, e)

feat_df = pd.DataFrame(rows).fillna(np.nan)
qc_df = pd.DataFrame(qc_rows)
feat_csv = os.path.join(OUT_DIR, "features_master.csv")
qc_csv = os.path.join(OUT_DIR, "qc_master.csv")
feat_df.to_csv(feat_csv, index=False)
qc_df.to_csv(qc_csv, index=False)
print("Saved feature table:", feat_csv)
print("Saved QC table:", qc_csv)


Found 5 files.


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 16.86it/s]

Saved feature table: data/features_scored_4levels\features_master.csv
Saved QC table: data/features_scored_4levels\qc_master.csv


In [16]:
# Cell 6 - save numpy arrays and feature column names for ML convenience
meta_cols = ['file','player','n_samples','fs_used','trial_mean_of_channel_means','trial_peak_of_channel_peaks']
feature_cols = [c for c in feat_df.columns if c not in meta_cols]
X = feat_df[feature_cols].values.astype(float)
players = feat_df['player'].values.astype(str)
np.save(os.path.join(OUT_DIR, "X.npy"), X)
np.save(os.path.join(OUT_DIR, "players.npy"), players)
pd.Series(feature_cols).to_csv(os.path.join(OUT_DIR, "feature_columns.csv"), index=False)
print("Saved X.npy, players.npy, feature_columns.csv")
print("Feature count:", len(feature_cols))


Saved X.npy, players.npy, feature_columns.csv
Feature count: 188


In [17]:
# Cell 7 - quick sanity outputs: first rows and summary stats and per-feature NaN counts
print("Preview features (first 3 rows):")
display(feat_df)
print("\nPer-feature missing counts (top 20):")
print(feat_df.isna().sum().sort_values(ascending=False))
print("\nDescriptive stats for some composite feature candidates:")
cands = [c for c in feature_cols if "__rms" in c or "__peak" in c or "__mnf" in c or "coact" in c]
display(feat_df[cands].describe().T)


Preview features (first 3 rows):


,file,player,n_samples,fs_used,Rectus Femoris right__mean,Rectus Femoris right__std,Rectus Femoris right__rms,Rectus Femoris right__rms_rel,Rectus Femoris right__mav,Rectus Femoris right__mav_rel,...,TibilaisÂ Anterior__LR_mean_absdev1,TibilaisÂ Anterior__LR_coact_simple,Gastrocnemius__LR_mean_ratio,Gastrocnemius__LR_mean_absdev1,Gastrocnemius__LR_coact_simple,Hamstrings__LR_mean_ratio,Hamstrings__LR_mean_absdev1,Hamstrings__LR_coact_simple,trial_mean_of_channel_means,trial_peak_of_channel_peaks
0,data/preprocessed_ml\ChipShot_Jordan_1.csv,preprocessed_ml,6438,2148.1481,0.016057,0.031330,0.035205,0.039419,0.016057,0.017979,...,0.710900,0.025024,0.643680,0.356320,0.045294,0.523824,0.476176,0.080363,0.017509,0.893091
1,data/preprocessed_ml\ChipShot_Jordan_2.csv,preprocessed_ml,6438,2148.1481,0.007522,0.014249,0.016113,0.036250,0.007702,0.017327,...,0.787283,0.077160,0.248263,0.751737,0.049854,0.411766,0.588234,0.048789,0.013238,0.444486
2,data/preprocessed_ml\ChipShot_Jordan_3.csv,preprocessed_ml,6438,2148.1481,0.008544,0.013870,0.016291,0.032449,0.008544,0.017019,...,0.535486,0.060854,0.240962,0.759038,0.030897,0.522268,0.477732,0.048276,0.014686,0.502041
3,data/preprocessed_ml\ChipShot_Jordan_4.csv,preprocessed_ml,6438,2148.1481,0.006413,0.010276,0.012113,0.066094,0.006413,0.034994,...,0.440790,0.062384,0.462724,0.537276,0.080515,0.601001,0.398999,0.058705,0.008882,0.183268
4,data/preprocessed_ml\ChipShot_Jordan_5.csv,preprocessed_ml,6438,2148.1481,0.009428,0.018646,0.020894,0.099692,0.009428,0.044983,...,0.680287,0.091244,0.311796,0.688204,0.069969,0.326196,0.673804,0.037257,0.010325,0.209587



Per-feature missing counts (top 20):
file                                 0
Gastrocnemius right__iEMG            0
TibilaisÂ Anterior left__iEMG        0
TibilaisÂ Anterior left__iEMG_rel    0
TibilaisÂ Anterior left__mnf         0
                                    ..
Hamstrings right__mrms_mean_rel      0
Hamstrings right__mrms_peak          0
Hamstrings right__mrms_peak_rel      0
Hamstrings left__mean                0
trial_peak_of_channel_peaks          0
Length: 194, dtype: int64

Descriptive stats for some composite feature candidates:


,count,mean,std,min,25%,50%,75%,max
Rectus Femoris right__rms,5.0,0.020123,0.008986,0.012113,0.016113,0.016291,0.020894,0.035205
Rectus Femoris right__rms_rel,5.0,0.054781,0.028386,0.032449,0.036250,0.039419,0.066094,0.099692
Rectus Femoris right__peak,5.0,0.092886,0.045842,0.053863,0.062246,0.076228,0.104894,0.167201
Rectus Femoris right__peak_rel,5.0,0.255416,0.150419,0.123987,0.171497,0.187216,0.293901,0.500477
Rectus Femoris right__mnf,5.0,4.165781,0.095440,4.033768,4.126252,4.176529,4.199259,4.293097
Rectus Femoris left__rms,5.0,0.011203,0.007046,0.006144,0.006439,0.007075,0.013937,0.022419
Rectus Femoris left__rms_rel,5.0,0.028086,0.012399,0.015606,0.015917,0.030724,0.033527,0.044656
Rectus Femoris left__peak,5.0,0.050261,0.039009,0.019598,0.024710,0.024818,0.075082,0.107096
Rectus Femoris left__peak_rel,5.0,0.116313,0.061178,0.055836,0.084070,0.093507,0.134832,0.213322
Rectus Femoris left__mnf,5.0,4.059651,0.249800,3.764832,3.819352,4.158818,4.271399,4.283854
